# PTDF with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Sourabh Dalvi

## Introduction

PowerSimulations.jl supports linear PTDF optimal power flow formulation. This example shows a
single multi-period optimization of economic dispatch with a linearized DC-OPF representation of
using PTDF power flow and how to extract duals values or locational marginal prices for energy.

## Dependencies
We can use the same RTS data and some of the initialization as in
[OperationsProblem example](../../notebook/3_PowerSimulations_examples/1_operations_problems.ipynb)
by sourcing it as a dependency.

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(
    joinpath(pkgpath, "test", "3_PowerSimulations_examples", "01_operations_problems.jl"),
);

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages

Since we'll be retrieving duals, we need a solver that returns duals values
here we use Ipopt.

In [2]:
using Ipopt
solver = optimizer_with_attributes(Ipopt.Optimizer)

MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[])

In the [OperationsProblem example](../../notebook/3_PowerSimulations_examples/1_operations_problems.ipynb)
we defined a unit-commitment problem with a copper plate representation of the network.
Here, we want do define an economic dispatch (linear generation decisions) with
linear DC-OPF using PTDF network representation.
So, starting with the network, we can select from _almost_ any of the endpoints on this
tree:

In [3]:
TypeTree(PSI.PM.AbstractPowerModel,  init_expand = 10, scopesep="\n")

PowerModels
AbstractPowerModel
├──PowerModels
│  AbstractACRModel
│  ├──ACRPowerModel
│  └──PowerModels
│     AbstractIVRModel
│     └──PowerModels
│        IVRPowerModel (0 children)
├──PowerModels
│  AbstractACPModel
│  └──ACPPowerModel
├──PowerModels
│  AbstractConicModel
│  ├──PowerModels
│  │  AbstractWRConicModel
│  │  └──PowerModels
│  │     AbstractSOCWRConicModel (1 children)
│  └──PowerModels
│     AbstractWRMModel
│     └──PowerModels
│        AbstractSDPWRMModel (2 children)
├──PowerModels
│  AbstractWRModel
│  ├──PowerModels
│  │  AbstractQCWRModel
│  │  ├──PowerModels
│  │  │  AbstractQCRMPowerModel (1 children)
│  │  └──PowerModels
│  │     AbstractQCLSModel (1 children)
│  └──PowerModels
│     AbstractSOCWRModel
│     └──SOCWRPowerModel (0 children)
├──PowerModels
│  AbstractLPACModel
│  └──PowerModels
│     AbstractLPACCModel
│     └──LPACCPowerModel (0 children)
├──PowerModels
│  AbstractBFModel
│  ├──PowerModels
│  │  AbstractBFAModel
│  │  └──PowerModels
│  │     BFAPowerModel (0 children)
│  ├──PowerModels
│  │  AbstractBFConicModel
│  │  └──PowerModels
│  │     AbstractSOCBFConicModel (1 children)
│  └──PowerModels
│     AbstractBFQPModel
│     └──PowerModels
│        AbstractSOCBFModel (1 children)
├──PowerModels
│  AbstractActivePowerModel
│  ├──PowerModels
│  │  AbstractDCPModel
│  │  ├──PowerModels
│  │  │  AbstractDCPLLModel (1 children)
│  │  ├──StandardPTDFModel (0 children)
│  │  ├──PowerModels
│  │  │  AbstractNFAModel (1 children)
│  │  ├──PowerModels
│  │  │  AbstractDCMPPModel (1 children)
│  │  └──DCPPowerModel (0 children)
│  ├──CopperPlatePowerModel
│  └──AreaBalancePowerModel
└──PowerModels
   AbstractACTModel
   └──ACTPowerModel

For now, let's just choose a standard PTDF formulation.

In [4]:
ed_template = template_economic_dispatch(network = StandardPTDFModel)


Operations Problem Specification

  transmission:  StandardPTDFModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = ThermalRampLimited
      DistRE:
        device_type = RenewableFix
        formulation = FixedOutput
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchReservoirBudget
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
     

Currently  energy budget data isn't stored in the RTS-GMLC dataset.

In [5]:
ed_template.devices[:Hydro] = DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver)

DeviceModel{HydroEnergyReservoir,HydroDispatchRunOfRiver}(HydroEnergyReservoir, HydroDispatchRunOfRiver, nothing, ServiceModel[])

Calculate the PTDF matrix.

In [6]:
PTDF_matrix = PTDF(sys)

PowerNetworkMatrix
:
  0.436221     -0.506679     0.0955772    …   0.0139213    0.0168526
  0.242695      0.220093    -0.199576        -0.0291078   -0.0352191
  0.321083      0.286586     0.103999         0.0151865    0.0183664
  0.240805      0.269317     0.029752         0.00430723   0.00522632
  0.195416      0.224003     0.0658252        0.00961407   0.0116263
  0.0884399     0.0729336    0.422869     …   0.0616101    0.0745751
  0.154255      0.14716      0.377554        -0.0907179   -0.109794
  0.240805      0.269317     0.029752         0.00430723   0.00522632
  0.321083      0.286586     0.103999         0.0151865    0.0183664
  0.195416      0.224003     0.0658252        0.00961407   0.0116263
  ⋮                                       ⋱               
 -0.00640688   -0.00621149  -0.0125463       -0.170422    -0.0870865
 -0.00640688   -0.00621149  -0.0125463       -0.170422    -0.0870865
 -0.0101178    -0.00980923  -0.0198131        0.129136    -0.137527
 -0.0101178    -0.00980

Now we can build a 4-hour economic dispatch / PTDF problem with the RTS data.
Here, we have to pass the keyword argument `constraint_duals` to OperationsProblem
with the name of the constraint for which duals are required for them to be returned in the results.

In [7]:
problem = OperationsProblem(
    EconomicDispatchProblem,
    ed_template,
    sys,
    horizon = 4,
    optimizer = solver,
    balance_slack_variables = true,
    constraint_duals = [:CopperPlateBalance, :network_flow],
    PTDF = PTDF_matrix,
)

┌ Info: Unit System changed to SYSTEM_BASE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/base.jl:287
┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/uqxkd/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: Data doesn't contain generators with ramp limits, consider adjusting your formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/uqxkd/src/devices_models/devices/thermal_generation.jl:505
┌ Warning: The data doesn't include devices of type Transformer2W, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/uqxkd/src/devices_models/device_constructors/common/constructor_validations.jl:3



Operations Problem Specification

  transmission:  StandardPTDFModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = ThermalRampLimited
      DistRE:
        device_type = RenewableFix
        formulation = FixedOutput
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
        de

And solve the problem and collect the results

In [8]:
res = solve!(problem);

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   128292
Number of nonzeros in inequality constraint Jacobian.:     1748
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     2724
                     variables with only lower bounds:      584
                variables with lower and upper bounds:     1656
                     variables with only upper bounds:        0
Total number of equality constraints.................:      776
Total number of inequality constraints...............:     1772
        inequality constraints with only lower bounds:      788
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      984

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

Here we collect the dual values from the results for the `CopperPlateBalance` and `network_flow`
constraints. In the case of PTDF network formulation we need to compute the final LMP for each bus in the system by
subtracting the duals (μ) of `network_flow` constraint multiplied by the PTDF matrix
from the  dual (λ) of `CopperPlateBalance` constraint.
Note:we convert the results from DataFrame to Array for ease of use.

In [9]:
λ = convert(Array, res.dual_values[:CopperPlateBalance])
μ = convert(Array, res.dual_values[:network_flow])

4×120 Array{Float64,2}:
 7.0463e-7   -1.29409e-7  4.95498e-6  3.14463e-6  …  -1.25954e-6  -2.86259e-6
 7.60597e-7  -3.13817e-7  4.90393e-6  3.07754e-6     -1.07287e-6  -2.45536e-6
 7.96643e-7  -4.36166e-7  4.85931e-6  3.02045e-6     -9.46744e-7  -2.17358e-6
 8.29101e-7  -5.66171e-7  4.77752e-6  2.93175e-6     -8.14169e-7  -1.87254e-6

Here we create Dict to store the calculate congestion component of the LMP which is a product of μ and the PTDF matrix.

In [10]:
buses = get_components(Bus, sys)
congestion_lmp = Dict()
for bus in buses
    congestion_lmp[get_name(bus)] = μ * PTDF_matrix[:, get_number(bus)]
end
congestion_lmp = DataFrame(congestion_lmp)

,Abel,Adams,Adler,Agricola,Aiken,Alber,Alder
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-3.41569e-8,-9.06212e-7,-2.01344e-7,-3.9962e-6,-3.85315e-6,-6.95364e-6,9.60362e-6
2,-3.87865e-7,-1.31838e-6,-2.69864e-7,-4.35011e-6,-4.18129e-6,-7.33061e-6,9.56838e-6
3,-6.53542e-7,-1.62258e-6,-3.40933e-7,-4.60204e-6,-4.41636e-6,-7.63447e-6,9.55733e-6
4,-1.07537e-6,-2.08096e-6,-5.30728e-7,-4.97512e-6,-4.77263e-6,-8.10939e-6,9.65538e-6


Finally here we get the LMP for each node in a lossless DC-OPF using the PTDF formulation.

In [11]:
LMP = λ .- congestion_lmp

,Abel,Adams,Adler,Agricola,Aiken,Alber,Alder,Alger,Ali
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6
2,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6
3,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6
4,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*